In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from functools import cache
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

Set some nice defaults

In [ ]:
def set_defaults(labelsize=12, dpi=250):
    mpl.rcParams["mathtext.fontset"] = "stix"
    mpl.rcParams["font.family"] = "STIXGeneral"
    mpl.rcParams["text.usetex"] = True
    plt.rc("xtick", labelsize=labelsize)
    plt.rc("ytick", labelsize=labelsize)
    plt.rc("axes", labelsize=labelsize)
    mpl.rcParams["figure.dpi"] = dpi


def set_grids(
    ax,
    minorticks=True,
    grid=False,
    bottom=True,
    left=True,
    right=True,
    top=True,
):
    if minorticks:
        ax.minorticks_on()

    ax.tick_params(
        which="both",
        direction="in",
        bottom=bottom,
        left=left,
        top=top,
        right=right,
    )

    if grid:
        ax.grid(which="minor", alpha=0.2, linestyle=":")
        ax.grid(which="major", alpha=0.5)

In [ ]:
set_defaults()

Simple helpers.

In [ ]:
def read_json(path):
    with open(path, 'r') as infile:
        dat = json.load(infile)
    return dat

In [ ]:
def load(root):
    root = Path(root)
    config = read_json(root / "config.json")
    results = read_json(root / "results.json")
    diagnostics_path = root / "diagnostics.json"
    diagnostics = None
    if diagnostics_path.exists():
        diagnostics = read_json(diagnostics_path)
    return config, results, diagnostics

Other helpers for nice plots.

In [ ]:
plot_kwargs = {
    'linewidth': 1.0,
    'marker': 's',
    'ms': 1.0,
    'capthick': 0.3,
    'capsize': 2.0,
    'elinewidth': 0.3
}

# Functions to make the visuals

Edit these however you want to affect how the report looks in the end!

## Arcsine law

In [ ]:
from scipy.integrate import quad

@cache
def hxw(x, w=0.5):
    """Equation 3 in the Cammarota 2018 paper."""

    def integrand(u):
        return 1.0 / (1.0 + u) / u**x

    return quad(integrand, w, np.inf)[0] * np.sin(np.pi * x) / np.pi

## Energy

In [ ]:
def make_energy_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
    plot_guideline=True,
    guideline_color=None,
    guideline_label=None,
):
    key = "energy"
    grid = np.array(results["grids"][key])[::every] + 1
    obs = results[key]
    mean = np.array(obs["mean"])[::every]
    if use_standard_error:
        err = np.array(obs["standard_error"])[::every]
    else:
        err = np.array(obs["standard_deviation"])[::every]

    ax.errorbar(
        grid,
        mean,
        yerr=2*err,
        color=color,
        label=label,
        **plot_kwargs
    )

    if plot_guideline:
        guideline_grid = grid[10:] * 5
        guideline = -1.0/config["beta"] * np.log(guideline_grid)
        guideline_color = guideline_color if guideline_color is not None else color
        
        diff = -1.0/config["beta"] * np.log(grid[-1]) - mean[-1]
        ax.plot(
            guideline_grid,
            guideline - diff,
            color=guideline_color,
            linestyle="-",
            label=guideline_label,
            linewidth=0.2
        )

    ax.set_xscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$E(t)$")
    
    set_grids(ax)


## Ridge

In [ ]:
def make_ridge_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
):

    key = "ridge_E"
    grid = np.array(results["grids"]["energy"])[::every] + 1
    obs = results[key]
    mean = np.array(obs["median_mean"])[::every]
    try:
        err = np.array(obs["median_std_err"])[::every]
    except:
        # Stupid inconsistency between two versions of hdspin
        # my bad
        err = np.array(obs["median_standard_error"])[::every]
    # mean = np.array(obs["mean_mean"])[::every]
    # mean_standard_error = np.array(obs["mean_standard_error"])[::every]
    
    # For ridges, we only want to start plotting after the first ridge
    # is recorded
    
    where = np.where(mean != 0.0)[0]

    ax.errorbar(
        grid[where],
        mean[where],
        yerr=2*err[where],
        color=color,
        label=label,
        **plot_kwargs
    )    
    
    ax.set_xscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$E_\mathrm{median~ridge}(t)$")
    
    set_grids(ax)

## Emax

In [ ]:
def make_emax2_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
):
    key = "emax"
    
    grid = np.array(results["grids"]["pi2"])[::every] + 1
    obs = results[key]
    mean = np.array(obs["mean"])[::every]
    if use_standard_error:
        err = np.array(obs["standard_error"])[::every]
    else:
        err = np.array(obs["standard_deviation"])[::every]

    ax.errorbar(
        grid,
        mean,
        yerr=2*err,
        color=color,
        label=label,
        **plot_kwargs
    )
    
    ax.set_xscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$\max_{t' \in [t/2, t]} E(t')$")
    
    set_grids(ax)

## Cache size

In [ ]:
def make_cache_size_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
    plot_guideline=True,
    guideline_color=None,
    guideline_label=None,
):
    key = "cache_size"
    grid = np.array(results["grids"]["energy"])[::every] + 1
    obs = results[key]
    mean = np.array(obs["mean"])[::every] / config["memory"]
    if use_standard_error:
        err = np.array(obs["standard_error"])[::every] / config["memory"]
    else:
        err = np.array(obs["standard_deviation"])[::every] / config["memory"]
        
    ax.errorbar(
        grid,
        mean,
        yerr=err,
        label=label,
        color=color,
        **plot_kwargs
    )
    
    if plot_guideline:
        guideline_grid = grid[10:] * 5
        guideline = guideline_grid**(1.0/config["beta"])
        guideline_color = guideline_color if guideline_color is not None else color
        
        diff = grid[-1]**(1.0/config["beta"]) / mean[-1]
        
        ax.plot(
            guideline_grid,
            guideline / diff,
            color=guideline_color,
            linestyle="-",
            label=guideline_label,
            linewidth=0.2
        )
    
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"Cache Saturation")
    
    set_grids(ax)

## Psi Config

In [ ]:
def make_psi_config_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
    plot_guideline=True,
    guideline_color=None,
    guideline_label=None,
):

    key = "psi_config"
    obs = results[key]
    mean = np.array(obs["mean"])
    standard_deviation = np.array(obs["standard_deviation"])
    standard_error = np.array(obs["standard_error"])
    grid = np.array(results["grids"]["psi"])
    where = np.where(mean != 0)[0]
    grid = grid[where]
    mean = mean[where]
    standard_deviation = standard_deviation[where]
    standard_error = standard_error[where]
    if use_standard_error:
        err = standard_error
    else:
        err = standard_deviation
        
    ax.errorbar(
        grid,
        mean,
        yerr=err,
        label=label,
        color=color,
        **plot_kwargs
    )
    
    if plot_guideline:
        exp_ratio = config["beta_critical"] / config["beta"]
        
        guideline = grid**(-exp_ratio)
        guideline_color = guideline_color if guideline_color is not None else color
        n = len(grid) // 2
        diff = grid[n]**(-exp_ratio) / mean[n]
        
        ax.plot(
            grid,
            guideline / diff,
            color="black",
            linewidth=0.2,
            # linestyle="--",
            # label=r"$\tau^{-\beta_\mathrm{c}/\beta}$"
        )

    
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$\psi_\mathrm{C}(t)$")
    
    set_grids(ax)

## Psi Basin

In [ ]:
def make_psi_basin_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
    plot_guideline=True,
    guideline_color=None,
    guideline_label=None,
):

    key = "psi_basin_E"
    obs = results[key]
    mean = np.array(obs["mean"])
    standard_deviation = np.array(obs["standard_deviation"])
    standard_error = np.array(obs["standard_error"])
    grid = np.array(results["grids"]["psi"])
    where = np.where(mean != 0)[0]
    grid = grid[where]
    mean = mean[where]
    standard_deviation = standard_deviation[where]
    standard_error = standard_error[where]
    if use_standard_error:
        err = standard_error
    else:
        err = standard_deviation
        
    ax.errorbar(
        grid,
        mean,
        yerr=err,
        label=label,
        color=color,
        **plot_kwargs
    )
    
    if plot_guideline:
        exp_ratio = config["beta_critical"] / config["beta"]
        
        guideline = grid**(-exp_ratio)
        guideline_color = guideline_color if guideline_color is not None else color
        n = len(grid) // 2
        diff = grid[n]**(-exp_ratio) / mean[n]
        
        ax.plot(
            grid,
            guideline / diff,
            color="black",
            linewidth=0.2,
            # linestyle="--",
            # label=r"$\tau^{-\beta_\mathrm{c}/\beta}$"
        )

    
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$\psi_\mathrm{B}(t)$")
    
    set_grids(ax)

## Aging Config

In [ ]:
def make_aging_config_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
    plot_guideline=True,
    guideline_color=None,
    guideline_label=None,
):
    
    key = "aging_config"
    obs = results[key]
    mean = np.array(obs["mean"])[::every]
    standard_error = np.array(obs["standard_error"])[::every]
    standard_deviation = np.array(obs["standard_deviation"])[::every]
    grid = np.array(results["grids"]["pi1"])[::every]
    if use_standard_error:
        err = standard_error
    else:
        err = standard_deviation
    
    ax.errorbar(
        grid,
        mean,
        yerr=err,
        label=label,
        color=color,
        **plot_kwargs
    )
    
    if plot_guideline:
        h = hxw(config["beta_critical"]/config["beta"])
        if h > 0:
            ax.axhline(h, linewidth=0.2, color="black")
    
    ax.set_xscale("log")
    # ax.set_yscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$\Pi_\mathrm{C}(t)$")
    
    set_grids(ax)

## Aging Basin

In [ ]:
def make_aging_basin_plot(
    ax,
    config,
    results,
    every=4,
    use_standard_error=True,
    plot_kwargs=plot_kwargs,
    color="blue",
    label=None,
    plot_guideline=True,
    guideline_color=None,
    guideline_label=None,
):
    
    key = "aging_basin_E"
    obs = results[key]
    mean = np.array(obs["mean"])[::every]
    standard_error = np.array(obs["standard_error"])[::every]
    standard_deviation = np.array(obs["standard_deviation"])[::every]
    grid = np.array(results["grids"]["pi1"])[::every]
    if use_standard_error:
        err = standard_error
    else:
        err = standard_deviation
    
    ax.errorbar(
        grid,
        mean,
        yerr=err,
        label=label,
        color=color,
        **plot_kwargs
    )
    
    if plot_guideline:
        h = hxw(config["beta_critical"]/config["beta"])
        if h > 0:
            ax.axhline(h, linewidth=0.2, color="black")
    
    ax.set_xscale("log")
    # ax.set_yscale("log")
    ax.set_xlabel(r"$t$")
    ax.set_ylabel(r"$\Pi_\mathrm{B}(t)$")
    
    set_grids(ax)

# Make report

In [ ]:
betas = [0.667, 1.333, 2.000, 3.000, 4.000]
N = 64
cmap = mpl.colormaps["viridis"].resampled(len(betas))

## Energy

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    guideline_label = None
    plot_guideline = True
    if ii == 1:
        guideline_label = r"$-T\log t$"
    elif ii == 0:
        plot_guideline = False
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_energy_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
        plot_guideline=plot_guideline,
        guideline_color="black",
        guideline_label=guideline_label
    )
ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()

## Ridge Energy

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))




for ii, beta in enumerate(betas):
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    
    if ii == 0:
        # Eth is the same for all beta
        et = config["energetic_threshold"]
        ax.axhline(et, color="black", linewidth=0.2, label="$E_\mathrm{th}=%.02f$" % et)
    
    make_ridge_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
    )
ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")



plt.show()

## Emax/2

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_emax2_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
    )
ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()

## Cache size

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    guideline_label = None
    plot_guideline = True
    if ii == 1:
        guideline_label = r"$t^T$"
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_cache_size_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
        guideline_color="black",
        plot_guideline=plot_guideline,
        guideline_label=guideline_label,
    )
ax.axhline(1, color="black", zorder=-3)
ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()

## Psi Config

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_psi_config_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
        plot_guideline=True,
    )

ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()

## Psi Basin

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_psi_basin_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
        plot_guideline=True,
    )

ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()

## Aging Config

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_aging_config_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
        plot_guideline=True,
    )

ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()

## Aging Basin

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))

for ii, beta in enumerate(betas):
    config, results, diagnostics = load(f"mc_work/project/beta{beta:.03f}_N{N}")
    make_aging_basin_plot(
        ax,
        config,
        results,
        color=cmap(ii),
        label=beta,
        use_standard_error=True,
        plot_guideline=True,
    )

ax.legend(frameon=False, bbox_to_anchor=(1.0, 0.5), loc="center left")
plt.show()